In [1]:
# Details:
# Study: Brain Network Organization and Behavior
# Note: This notebook processes the frustration condition for all subjects usint the Power 264 atlas


# Credits:
# Written by Richard Huskey
# https://nilearn.github.io/auto_examples/03_connectivity/plot_group_level_connectivity.html#sphx-glr-auto-examples-03-connectivity-plot-group-level-connectivity-py


# Notes and dependencies
# None

In [2]:
import numpy as np
import glob
from nilearn import datasets

/home/huskeyadmin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Import one subject per line
# NOTE: sub count starts at 005 (subs 001-004 were pilot scans using a different procedure)
# NOTE: sub 010 excluded due to abnormal radiological reading
# NOTE: subs 027 and 038 exhibited contraindication to scanning and therefore are excluded
# NOTE: subs 022 and 035 were excluded since they failed the behavioral compliance check
# NOTE: subs 013, 018, and 019 were excluded since they were outliers in a group-level MRIQC analysis

filenames = sorted(glob.glob('nii/*.gz'))

for f in filenames:
    print(f)

nii/sub-005_task-game_run-03_bold_space-MNI152NLin2009cAsym_preproc_merged.nii.gz
nii/sub-006_task-game_run-01_bold_space-MNI152NLin2009cAsym_preproc_merged.nii.gz
nii/sub-007_task-game_run-03_bold_space-MNI152NLin2009cAsym_preproc_merged.nii.gz
nii/sub-008_task-game_run-02_bold_space-MNI152NLin2009cAsym_preproc_merged.nii.gz
nii/sub-009_task-game_run-02_bold_space-MNI152NLin2009cAsym_preproc_merged.nii.gz
nii/sub-011_task-game_run-01_bold_space-MNI152NLin2009cAsym_preproc_merged.nii.gz
nii/sub-012_task-game_run-02_bold_space-MNI152NLin2009cAsym_preproc_merged.nii.gz
nii/sub-014_task-game_run-01_bold_space-MNI152NLin2009cAsym_preproc_merged.nii.gz
nii/sub-015_task-game_run-01_bold_space-MNI152NLin2009cAsym_preproc_merged.nii.gz
nii/sub-016_task-game_run-02_bold_space-MNI152NLin2009cAsym_preproc_merged.nii.gz
nii/sub-017_task-game_run-02_bold_space-MNI152NLin2009cAsym_preproc_merged.nii.gz
nii/sub-020_task-game_run-03_bold_space-MNI152NLin2009cAsym_preproc_merged.nii.gz
nii/sub-021_task

In [4]:
from nilearn import datasets

In [5]:
# This imports the Power 2011 atlas with 264 cortical and subcortical ROIs

power = datasets.fetch_coords_power_2011()
print('Power atlas comes with {0}.'.format(power.keys()))

Power atlas comes with dict_keys(['rois', 'description']).


In [6]:
from nilearn import input_data

In [7]:
# Makes a table with the x,y,z coordinates of each seed ROI

coords = np.vstack((power.rois['x'], power.rois['y'], power.rois['z'])).T

print('Stacked power coordinates in array of shape {0}.'.format(coords.shape))

# The code below exports the ROI x,y,z coordinates of each seed ROI to a text file

np.savetxt('adjacency_matrix/power_rois.txt',coords)

Stacked power coordinates in array of shape (264, 3).


In [8]:
# Now, lets make some masks
# Note, these are 5mm spheres so that ROIs do not overlap
# They also highpass filter the data since this wasn't done in fmriprep (filter = 250s)

masker = input_data.NiftiSpheresMasker(
    seeds=coords, smoothing_fwhm=None, radius=5.,
    detrend=True, high_pass=.004, t_r=2.0, memory='nilearn_cache', memory_level=1)

In [ ]:
# This creates a timeseries matrix for each subect for each run

pooled_subjects = []

for f in filenames:
    time_series = masker.fit_transform(f)
    pooled_subjects.append(time_series)

In [ ]:
# If files are being imported correctly, there should 120 volumes reported here

print ('timeseries has {0} samples. This is how many volumes there are in the functional run'.format(time_series.shape[0]))

In [ ]:
# Now, lets estimate some connectivities using correlation

from nilearn.connectome import ConnectivityMeasure

In [ ]:
correlation_measure = ConnectivityMeasure(kind='correlation')

In [ ]:
# Compute individual correlation matricies for each subject

correlation_matrices = correlation_measure.fit_transform(pooled_subjects)

In [ ]:
# Shape should be (n, 264, 264) where n = the number of subjects/runs in your study

print('Correlations of subject runs are stacked in an array of shape {0}'
      .format(correlation_matrices.shape))

In [ ]:
# Calculates a mean (group-level) correlation matrix
# This tells you the shape of your mean correlation matrix
# Shape should just be (264, 264) because you are taking the average correlation of all subjects here

mean_correlation_matrix = correlation_measure.mean_
print('Mean correlation has shape {0}.'.format(mean_correlation_matrix.shape))

In [ ]:
# Set the diagonal to zero (This facilitates the Fishers Z Transform in the next step)

np.fill_diagonal(mean_correlation_matrix, 0)

In [ ]:
# Fishers Z Transform the average (group-level) matrix

fisherz_mean_correlation_matrix = np.arctanh(mean_correlation_matrix)

In [ ]:
# This saves the Fisher Z transformed correlation matrix as a txt output in the current working directory

np.savetxt('adjacency_matrix/frust_cond_corel_matrix.txt',fisherz_mean_correlation_matrix)

In [ ]:
# Now, lets estimate some connectivities using covariance

covariance_measure = ConnectivityMeasure(kind='covariance')

In [ ]:
covariance_matricies = covariance_measure.fit_transform(pooled_subjects)

In [ ]:
# Shape should be (n, 264, 264) where n = the number of subjects/runs in your study

print('Covariances of subject runs are stacked in an array of shape {0}'
      .format(covariance_matricies.shape))

In [ ]:
# This tells you the shape of your mean covariance matrix
# Shape should just be (264, 264) because you are taking the average correlation of all subjects here

mean_covariance_matrix = covariance_measure.mean_
print('Mean covariance has shape {0}.'.format(mean_covariance_matrix.shape))

In [ ]:
# This saves the covariance matrix as a txt output in the current working directory

np.savetxt('adjacency_matrix/frust_cond_covar_matrix.txt',mean_covariance_matrix)

In [ ]:
# Sets the diagonal to zero for each subject's individual correlation matrix

for j in range(len(correlation_matrices)):
    np.fill_diagonal(correlation_matrices[j], 0)

In [ ]:
# Fishers Z transforms each subject's correlation matrix

fisherz_correlation_matrices = np.arctanh(correlation_matrices)

In [ ]:
# This saves the correlation matrix for each subject as a txt output in the current working directory

for i in range(len(fisherz_correlation_matrices)):
    np.savetxt('adjacency_matrix/frust_cond_corel_matrix_%i.txt'%i, fisherz_correlation_matrices[i] )